In [10]:
#import libraries
import numpy as np

In [17]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [18]:
import pandas as pd

In [19]:
import nltk

In [20]:
from nltk.corpus import stopwords

In [21]:
import string

In [15]:
#Load the data
from google.colab import files
uploaded = files.upload()

Saving train_news.csv to train_news (2).csv


#Read the csv file


In [24]:
import io

In [25]:
df=pd.read_csv(io.BytesIO(uploaded['train_news.csv']))

In [44]:
df.shape

(20800, 6)

#print the first 5 rows

In [29]:

df.head()

,Unnamed: 0,id,headline,written_by,news,label
0,0,9653,Ethics Questions Dogged Agriculture Nominee as...,Eric Lipton and Steve Eder,"WASHINGTON — In Sonny Perdue’s telling, Geo...",0
1,1,10041,U.S. Must Dig Deep to Stop Argentina’s Lionel ...,David Waldstein,HOUSTON — Venezuela had a plan. It was a ta...,0
2,2,19113,Cotton to House: ’Do Not Walk the Plank and Vo...,Pam Key,"Sunday on ABC’s “This Week,” while discussing ...",0
3,3,6868,"Paul LePage, Besieged Maine Governor, Sends Co...",Jess Bidgood,"AUGUSTA, Me. — The beleaguered Republican g...",0
4,4,7596,A Digital 9/11 If Trump Wins,Finian Cunningham,Finian Cunningham has written extensively on...,1


In [42]:
df.isnull().sum()

Unnamed: 0       0
id               0
headline       558
written_by    1957
news            39
label            0
dtype: int64

In [50]:
df.dropna(inplace=True)

In [51]:
df.shape

(18285, 6)

Print the Columns

In [52]:
df.columns

Index(['Unnamed: 0', 'id', 'headline', 'written_by', 'news', 'label'], dtype='object')

In [57]:
df_final=df.drop(['Unnamed: 0', 'id', 'headline', 'written_by'], axis = 1 )

In [58]:
df_final.head()

,news,label
0,"WASHINGTON — In Sonny Perdue’s telling, Geo...",0
1,HOUSTON — Venezuela had a plan. It was a ta...,0
2,"Sunday on ABC’s “This Week,” while discussing ...",0
3,"AUGUSTA, Me. — The beleaguered Republican g...",0
4,Finian Cunningham has written extensively on...,1


In [59]:
df_final.isnull().sum()

news     0
label    0
dtype: int64

spam detection


In [60]:
#Downoad the stopword packages
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### Creating a function to convert the text in lowercase, remove the extra space, special chr., ulr and links.

In [61]:
df_final.shape

(18285, 2)

In [62]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [63]:
df_final["news"] = df_final["news"].apply(wordopt)

In [64]:
df_final.head(10)

,news,label
0,washington in sonny perdue s telling geo...,0
1,houston venezuela had a plan it was a ta...,0
2,sunday on abc s this week while discussing ...,0
3,augusta me the beleaguered republican g...,0
4,finian cunningham has written extensively on...,1
6,the state of new jersey says you can t eat the...,0
7,advocates say prison officials at the kilby co...,1
9,leave a reply toni emerson when we fall in ...,1
10,breaking clinton fixer reveals sick smear hil...,1
11,home politics us news bloomberg backed p...,1


#### Defining dependent and independent variable as x and y

In [66]:
x = df_final["news"]
y = df_final["label"]

#### Splitting the dataset into training set and testing set. 

In [67]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

#### Convert text to vectors

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [69]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

### 1. Logistic Regression

In [70]:
from sklearn.linear_model import LogisticRegression

In [71]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [72]:
pred_lr=LR.predict(xv_test)

In [73]:
LR.score(xv_test, y_test)

0.9507793273174734

In [74]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.95      0.96      0.96      2057
           1       0.95      0.93      0.94      1600

    accuracy                           0.95      3657
   macro avg       0.95      0.95      0.95      3657
weighted avg       0.95      0.95      0.95      3657



In [75]:
### 2. Decision Tree Classification

In [76]:
from sklearn.tree import DecisionTreeClassifier

In [77]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [78]:
pred_dt = DT.predict(xv_test)

In [79]:
DT.score(xv_test, y_test)

0.8881596937380366

In [80]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.90      0.91      0.90      2057
           1       0.88      0.86      0.87      1600

    accuracy                           0.89      3657
   macro avg       0.89      0.89      0.89      3657
weighted avg       0.89      0.89      0.89      3657



### 3. Gradient Boosting Classifier

In [81]:
from sklearn.ensemble import GradientBoostingClassifier

In [82]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [83]:
pred_gbc = GBC.predict(xv_test)

In [84]:
GBC.score(xv_test, y_test)

0.9310910582444627

In [85]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      2057
           1       0.92      0.92      0.92      1600

    accuracy                           0.93      3657
   macro avg       0.93      0.93      0.93      3657
weighted avg       0.93      0.93      0.93      3657



### 4. Random Forest Classifier

In [86]:
from sklearn.ensemble import RandomForestClassifier

In [87]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [88]:
pred_rfc = RFC.predict(xv_test)

In [89]:
RFC.score(xv_test, y_test)

0.9094886519004649

In [90]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.87      0.98      0.92      2057
           1       0.97      0.82      0.89      1600

    accuracy                           0.91      3657
   macro avg       0.92      0.90      0.91      3657
weighted avg       0.92      0.91      0.91      3657



In [91]:
#Logistic regression gives the best result of accuracy 95%